In [23]:
import pandas as pd
import os, json
import requests

Import and clean the data

In [40]:
name_list=os.listdir('raw_data')

In [42]:
def get_data_from_json(file_name):

    json_file = f'raw_data/{file_name}'
    with open(json_file,'r') as f:
        all_data=json.load(f)
    data=all_data['items']  
    df = pd.DataFrame.from_records(data)[['id','word','rate', 'username', 'sex', 'code',
                                          'country','num_votes', 'num_positive_votes', 'pathmp3']]
    return df

In [59]:
df_list=[]
df_list_one_mp3=[]
for file in name_list:
    df=get_data_from_json(file)
    
    #filter only for UK
    #df=df.loc[df['country'] == 'United Kingdom'] 
    
    #get top two results
    df=df.head(2)
    if df.shape[0] == 2:
        
        #create list of dataframes
        df_list.append(df)
    elif df.shape[0] == 1:
        df_list_one_mp3.append(df)
    
#merge dataframes
main_df = pd.concat(df_list)
main_df=main_df.reset_index(drop=True)
if len(df_list_one_mp3)!=0:
    single_mp3_df = pd.concat(df_list_one_mp3)

Download the MP3 files

In [32]:
for index,row in main_df.iterrows():
    response = requests.get(row['pathmp3'])
    name=row['id']
    with open(f'{name}.mp3','wb') as f:
        f.write(response.content)